![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook demonstrates how to capture video from a V4L2 device and display the output on a monitor using a DRM/KMS display device. This notebook uses the GStreamer multimedia framework. In addition, the memory bandwidth is measured and plotted in a parallel notebook.

Three types of V4L2 devices are supported in this notebook:
* Virtual Video Test driver (vivid)
* USB Video Class (UVC) driver (usbcam)
* MIPI CSI-2 capture pipeline using the Leopard IMX274 FMC daughter card (mipi)
* HDMI Rx capture pipeline (hdmi)

See nb2 for details on the mentioned V4L2 devices.

The display device uses the Xilinx DRM/KMS driver. A video mixer supports alpha blending of multiple layers (also called planes). The plane formats are fixed and configured as follows:
* 4 RGB planes (IDs: 34-37)
* 4 YUY2 planes (IDs: 38-41)
* 1 ARGB plane (ID: 41) - this is the primary plane used for setting the CRTC resolution

The video mixer is connected to an HDMI encoder which drives the display. Both video mixer and HDMI encoder are implemented inside the FPGA.

The video pipeline is composed of the following GStreamer elements:
* The ``mediasrcbin`` element is used to capture video from a V4L2 device
* The ``perf`` element is used to measure and print the frame rate
* The ``kmssink`` element is used to display video on a monitor using the DRM/KMS kernel subsystem

The ``base-trd-apm`` notebook is executed in parallel to this notebook to measure and plot the memory bandwidth of the live video pipeline.

In this notebook, you will:
1. Create a GStreamer video pipeline that captures video from a V4L2 device and displays the video on a monitor using DRM/KMS.
2. Run the ``base-trd-apm`` notebook to measure and plot the memory bandwidth while the video pipeline is running.
3. Create a GStreamer pipeline graph and view it inside this notebook.

# 2. Imports and Initialization

Import all python modules required for this notebook. 

In [ ]:
from IPython.display import Image, display, clear_output
import glob
import subprocess
import pydot
import sys
import gi
gi.require_version('Gst', '1.0')
from gi.repository import GObject, GLib, Gst

This is the Base TRD notebook 3 (nb3).

In [ ]:
nb = "nb3"

Create a directory for saving the pipeline graph as dot file. Set the GStreamer debug dot directory environment variable to point to that directory.

In [ ]:
dotdir = "/home/root/gst-dot/" + nb
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

Initialize the GStreamer library. Enable debug by setting the debug string, set default to level 1 for all categories.

In [ ]:
Gst.init(None)
Gst.debug_set_threshold_from_string('*:1', True)

# 3. Run the APM Notebook to Plot the Memory Bandwidth

Open the ``base-trd-apm.ipynb`` notebook from the *File Browser* in a new tab. Execute the notebook by selecting *Run -> Run All Cells* from the Jupyter Lab menu bar. In section 4 of the APM notebook, a horizontal bar graph is shown that plots the currently consumed memory bandwidth split out by different AXI ports. For more information, read the APM notebook tutorial.

Once you see the graph, right-click the graph and select *Create New View for Output*. This will create a new window/tab with just the graph. Now re-arrange the window by dragging it to the the right side of the screen so it shows side-by-side with the notebook window (see screenshot below).

![APM Plot](images/apm-plot-nb3.jpg "APM Plot")

Switch tabs back to the nb3 notebook and follow the steps below. Once the video pipeline is running, you will notice the bar graph will be updated live with the measured memory bandwidth numbers in Gbps.

**Note:** You can keep the memory bandwith output view open while switching between notebooks. There is no need to restart the APM notebook.

# 4. Create and Configure the GStreamer Elements

The ``get_media_by_device`` function returns the matching media node for a given video capture source. The following sources are supported in this notebook:
* ``vivid`` : virtual video device (default)
* ``usb`` : requires USB webcam
* ``mipi`` : platform1 only, requires FMC card
* ``hdmi`` : platform3 only, requires HDMI input

In [ ]:
def get_media_dev_by_name(src):
    sources = {
        'vivid' : 'vivid',
        "usb" : 'uvcvideo',
        'mipi' : 'vcap_csi',
        'hdmi' : 'vcap_hdmi'
    }
    devices = glob.glob('/dev/media*')
    for dev in devices:
        proc = subprocess.run(['media-ctl', '-d', dev, '-p'], capture_output=True, encoding='utf8')
        for line in proc.stdout.splitlines():
            if sources[src] in line:
                return dev

Select the ``source`` based on available media devices for this platform. The default source is set to ``vivid``. Update the value next to the comment to select USB webcam or MIPI single-sensor if connected.

In [ ]:
source = "vivid" # Change source to vivid, usb, mipi, hdmi

media_device = get_media_dev_by_name(source) 
if media_device is None:
    raise Exception('Unable to find video source ' + source + '. Make sure the device is plugged in, powered, and the correct platform is used.')

The source pads of the ``mediasrcbin`` element are created dynamically when it detects the incoming stream. The ``pad-added`` signal is emitted and this ``pad_added`` callback function is executed. It links the source pads of the mediasrcbin elements to the sink pads of the ``caps`` elements.

Set the ``io-mode`` on the pad which propagates to the ``v4l2src`` node. If MIPI is selected, set the I/O mode to DMABUF (https://www.kernel.org/doc/html/v4.16/driver-api/dma-buf.html) which allows sharing of video buffers in 0-copy fashion between the source and sink elements. Otherwise, set the I/O mode to mmap.

In [ ]:
def pad_added(element, pad):
    sink_pad = caps.get_static_pad("sink")
    if not sink_pad.is_linked():
        pad.link(sink_pad)
        if source == "mipi" or source == "hdmi":
            pad.set_property("io-mode", "dmabuf")
        else:
            pad.set_property("io-mode", "mmap")

Create the ``mediasrcbin`` element which is a bin element that uses the standard ``v4l2src`` element inside. Set the following some properties:
* Set the ``media-device`` property to the desired media device node
* Register the above ``pad_added`` callback function with the ``pad-added`` signal of the ``mediasrcbin`` element.


In [ ]:
src = Gst.ElementFactory.make("mediasrcbin")
src.set_property("media-device", media_device)
src.connect("pad_added", pad_added);

Create a caps filter element to set the desired resolution (width and height) and format. The caps filter is configured to parse the mentioned properties from a string.

The default resolution is set to 1280x720 and the format to YUY2 as those are commonly supported by USB webcams.

If ``mipi`` or ``hdmi`` is selected as source type, the maximum supported resolution is 3840x2160 (4K) at 60 fps. Note that the connected monitor also needs to support this resolution, otherwise the pipeline will fail during caps negotiation (see modeprint output below).

In [ ]:
res_dict = {
    "720p" : (1280, 720),
    "1080p" : (1920, 1080),
    "2160p" : (3840, 2160)
}
res = "720p" # Change the resolution string to 720p, 1080p, or 2160p (mipi only)
width = res_dict[res][0]
height = res_dict[res][1]
print("Selected resolution: " + str(width) + "x" + str(height))

fmt = "YUY2"

cap_string = "video/x-raw, width=" + str(width) + ", height=" + str(height) + ", format=" + fmt
if source == "mipi" or source == "hdmi":
    fps = "60/1"
    cap_string = cap_string + ", framerate=" + fps

caps = Gst.ElementFactory.make("capsfilter")
cap = Gst.Caps.from_string(cap_string)
caps.set_property("caps", cap)

Create the ``perf`` element which is used to measure and print the frame rate while the video pipeline is running.

In [ ]:
perf = Gst.ElementFactory.make("perf")

The display driver creates a DRM device node with the module name ``xlnx``.

List information about the DRM device by passing the module name to the ``modeprint`` utility.

In [ ]:
!modeprint xlnx

Create the ``kmssink`` element and set some properties:
* Set the ``plane-id`` property to the ID value of the target plane. The default value 34 is set to the first YUY2 plane.
* Set the ``render-rectangle`` property to a quadruple consisting of x-offset, y-offset, width, and height. The render-rectangle allows moving a plane position on the display.

In [ ]:
plane_id = 38
xoff = 0 # Change this value to move the plane position in the x-direction
yoff = 0 # Change this value to move the plane position in the y-direction
render_rectangle = Gst.ValueArray((xoff, yoff, width, height))

sink = Gst.ElementFactory.make("kmssink")
sink.set_property("plane-id", plane_id)
sink.set_property("render-rectangle", render_rectangle)

# Uncomment the below code to read back the newly set property values
#print("sink properties: ")
#print("plane-id: " + str(sink.get_property("plane-id")))

# 5. Create and Run the GStreamer Pipeline

Create the pipeline, add all elements, and link them together.

In [ ]:
pipeline = Gst.Pipeline.new(nb)

pipeline.add(src)
pipeline.add(caps)
pipeline.add(perf)
pipeline.add(sink)

caps.link(perf)
perf.link(sink);

The ``bus_call`` function listens on the bus for ``EOS``, ``INFO`` and ``ERROR`` events. In case of ``EOS`` or ``ERROR``, stop the pipeline (set to ``NULL`` state) and quit the main loop. 

For ``INFO`` and ``ERROR`` events, parse and print the info/error message. The ``perf`` element generates ``INFO`` events with the measured frame rate.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.INFO:
        err, info = message.parse_info()
        sys.stderr.write("Info: %s\n" % info)
        clear_output(wait=True)
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

Start the pipeline (set to ``PLAYING`` state), create the main loop and listen to messages on the bus. Register the ``bus_call`` callback function with the ``message`` signal of the bus. Start the main loop.

The video will be displayed on the monitor. The frame rate will be printed and updated below the code cell.

To stop the pipeline, click the square shaped icon labelled 'Interrupt the kernel' in the top menu bar. Create a dot graph of the pipeline topology before stopping the pipeline. Quit the main loop.

In [ ]:
pipeline.set_state(Gst.State.PLAYING);

loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    pass

# 6. View the GStreamer Pipeline Graph

Register dot plugins for png export to work.

In [ ]:
!dot -c

Convert the dot file to png and display the pipeline graph. The image will be displayed below the following code cell. Double click on the generate image file to zoom in.

**Note:** This step may take a few seconds.

In [ ]:
dotfile = dotdir + "/" + nb + ".dot"
graph = pydot.graph_from_dot_file(dotfile, 'utf-8')
display(Image(graph[0].create(None, 'png', 'utf-8')))

# 7. Summary

In this notebook you learned how to:
1. Create a GStreamer pipeline that demonstrates how to capture video from a V4L2 device and display it on a monitor
2. Plot the live memory bandwidth by running the APM notebook in parallel
3. Export the pipeline topology as a dot file image and display it in the notebook

<center>Copyright© 2019 Xilinx</center>